In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from reservoirpy.nodes import Reservoir, Ridge, Input
import numpy as np
from reservoirpy.observables import nrmse, rsquare
import json
from reservoirpy.hyper import research
from reservoirpy.hyper import plot_hyperopt_report
from reservoirpy.observables import rmse, rsquare, nrmse

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

emg_data_train = pd.read_csv('./Kinematics_Train_Data_EMG.csv')
emg_data_test = pd.read_csv('./Kinematics_Test_Data_EMG.csv')
angles_data_train = pd.read_csv('./Kinematics_Train_Data_Angles.csv')
angles_data_test = pd.read_csv('./Kinematics_Test_Data_Angles.csv')

2022-04-22 17:14:36.288241: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-22 17:14:36.288264: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#angles_data_train = angles_data_train.drop(columns = ["Lead_Pelvis_Ant", "Lead_Pelvis_Olb", "Lead_Pelvis_IE","Trail_Pelvis_Ant", "Trail_Pelvis_Olb", "Trail_Pelvis_IE"])
#angles_data_test = angles_data_test.drop(columns = ["Lead_Pelvis_Ant", "Lead_Pelvis_Olb", "Lead_Pelvis_IE","Trail_Pelvis_Ant", "Trail_Pelvis_Olb", "Trail_Pelvis_IE"])
angles_data_test
len(emg_data_train)
emg_data_train_test = np.array(emg_data_train)
data_per_subject = 101
n_subjects = int (emg_data_train_test.shape[0]/data_per_subject)

emg_data_train_test = emg_data_train_test.reshape(n_subjects, data_per_subject, 16)

angles_data_train_test = np.array(angles_data_train)
angles_data_train_test = angles_data_train_test.reshape(n_subjects, data_per_subject, 20)

# how many output angles you want to predict
n_angles = 17
angles_data_train_test = angles_data_train_test[:, :, 0:n_angles]

emg_data_train_test_ev = np.array(emg_data_train)
emg_data_train_test_ev

emg_data_test_test = np.array(emg_data_test)
emg_data_test_test = emg_data_test_test.reshape(n_subjects, data_per_subject, 16)

#emg_data_test_test = emg_data_test_test.reshape(14, data_per_subject, 16)
emg_data_test_test

angles_data_test_test = np.array(angles_data_test)
#angles_data_test_test = emg_data_test_test.reshape(14, data_per_subject, 16)
angles_data_test_test = angles_data_test_test.reshape(n_subjects, data_per_subject, 20)
angles_data_test_test = angles_data_test_test[:, :, 0:n_angles]

# emg_left_01_test = np.array(emg_left_01)
# emg_left_01_test = emg_left_01_test.reshape(1, data_per_subject, 16)
# emg_left_01_test
# angles_left_01_test = np.array(angles_left_01)
# angles_left_01_test = angles_left_01_test.reshape(1, data_per_subject, 20)
# angles_left_01_test

x_train = emg_data_train_test
y_train = angles_data_train_test
x_test = emg_data_test_test
y_test = angles_data_test_test
x_train_ev = emg_data_train_test_ev

In [3]:
print (x_train.shape)
print (y_train.shape)
print (x_test.shape)
print (y_test.shape)
print (x_train_ev.shape)

(14, 101, 16)
(14, 101, 17)
(14, 101, 16)
(14, 101, 17)
(1414, 16)


In [4]:
batch_size = 1
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
input_dim = (data_per_subject, 16)

units = 512
output_size = 101  # labels are from 0 to 9

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    model = keras.models.Sequential(
        [
            #keras.layers.GRU(256, return_sequences=True),
            #keras.layers.RNN(keras.layers.LSTMCell(units)),
            keras.layers.SimpleRNN(units, input_shape=( 101, 16)),
            #keras.layers.Embedding(input_dim=101, output_dim=int((output_size*n_angles)/units)),
            keras.layers.Embedding(input_dim=101, output_dim=8),
            keras.layers.BatchNormalization(),
            tf.keras.layers.Conv1D(n_angles, units-(data_per_subject-1)),
            keras.layers.Dense(n_angles),
            #keras.layers.Reshape((101, n_angles), input_shape=(101*n_angles,)),
        ]
    )
    return model

model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 512)               270848    
                                                                 
 embedding (Embedding)       (None, 512, 8)            808       
                                                                 
 batch_normalization (BatchN  (None, 512, 8)           32        
 ormalization)                                                   
                                                                 
 conv1d (Conv1D)             (None, 101, 17)           56049     
                                                                 
 dense (Dense)               (None, 101, 17)           306       
                                                                 
Total params: 328,043
Trainable params: 328,027
Non-trainable params: 16
_________________________________________________

2022-04-22 17:14:37.493962: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-22 17:14:37.493983: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-22 17:14:37.493998: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kianoosh): /proc/driver/nvidia/version does not exist
2022-04-22 17:14:37.494197: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=keras.losses.MeanAbsoluteError(),
    optimizer="sgd",
    metrics=["accuracy"],
)


model.fit(
    x_train, y_train, batch_size=batch_size, epochs=200
)


Epoch 1/200
14/14 [==============================] - 1s 9ms/step - loss: 13.0175 - accuracy: 0.0000e+00
Epoch 2/200
14/14 [==============================] - 0s 9ms/step - loss: 13.0048 - accuracy: 0.0000e+00
Epoch 3/200
14/14 [==============================] - 0s 10ms/step - loss: 12.9868 - accuracy: 0.0000e+00
Epoch 4/200
14/14 [==============================] - 0s 10ms/step - loss: 12.9542 - accuracy: 0.0000e+00
Epoch 5/200
14/14 [==============================] - 0s 9ms/step - loss: 12.8847 - accuracy: 0.0000e+00
Epoch 6/200
14/14 [==============================] - 0s 9ms/step - loss: 12.7185 - accuracy: 0.0000e+00
Epoch 7/200
14/14 [==============================] - 0s 10ms/step - loss: 12.2933 - accuracy: 0.0000e+00
Epoch 8/200
14/14 [==============================] - 0s 9ms/step - loss: 11.3379 - accuracy: 0.0000e+00
Epoch 9/200
14/14 [==============================] - 0s 9ms/step - loss: 10.0607 - accuracy: 0.2723
Epoch 10/200
14/14 [==============================] - 0s 9ms/step

14/14 [==============================] - 0s 9ms/step - loss: 7.5626 - accuracy: 0.5898
Epoch 77/200
14/14 [==============================] - 0s 9ms/step - loss: 7.6000 - accuracy: 0.5898
Epoch 78/200
14/14 [==============================] - 0s 10ms/step - loss: 7.5566 - accuracy: 0.5898
Epoch 79/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5904 - accuracy: 0.5898
Epoch 80/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5754 - accuracy: 0.5898
Epoch 81/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5885 - accuracy: 0.5898
Epoch 82/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5751 - accuracy: 0.5898
Epoch 83/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5677 - accuracy: 0.5898
Epoch 84/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5903 - accuracy: 0.5898
Epoch 85/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5728 - accuracy: 0.5898
Epoch 86/200

14/14 [==============================] - 0s 9ms/step - loss: 7.5431 - accuracy: 0.5898
Epoch 158/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5712 - accuracy: 0.5898
Epoch 159/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5604 - accuracy: 0.5898
Epoch 160/200
14/14 [==============================] - 0s 10ms/step - loss: 7.4910 - accuracy: 0.5898
Epoch 161/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5622 - accuracy: 0.5898
Epoch 162/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5431 - accuracy: 0.5898
Epoch 163/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5471 - accuracy: 0.5898
Epoch 164/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5416 - accuracy: 0.5898
Epoch 165/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5180 - accuracy: 0.5898
Epoch 166/200
14/14 [==============================] - 0s 9ms/step - loss: 7.5546 - accuracy: 0.5898
Epo

In [11]:
mse = tf.keras.losses.MeanSquaredError()

mse_train = mse (model.predict(x_train), y_train)
mse_test = mse (model.predict(x_test), y_test)

# root mean squared error
rmse_train = mse_train**0.5
rmse_test = mse_test**0.5

print ("training MSE", mse_train)
print ("test MSE", mse_test)

print ("training RMSE", rmse_train)
print ("test RMSE", rmse_test)

training MSE tf.Tensor(122.2508316040039, shape=(), dtype=float64)
test MSE tf.Tensor(122.07171630859375, shape=(), dtype=float64)
training RMSE tf.Tensor(11.056709800117027, shape=(), dtype=float64)
test RMSE tf.Tensor(11.048606984982031, shape=(), dtype=float64)


In [12]:
model.predict(x_train).shape

(14, 101, 17)

In [8]:
y_train.shape

(14, 101, 17)

In [13]:
import numpy as np
import scipy.stats as stats

Array1 = y_train
Array2 = model.predict_on_batch(x_train)
resultArray = np.ones(n_subjects*data_per_subject*n_angles).reshape(n_subjects,data_per_subject,n_angles)
for i in range(Array1.shape[0]):
        for j in range(Array1.shape[1]):
            for k in range(Array1.shape[2]):
                slope, intercept, r_value, p_value, std_err = stats.linregress(
                    Array1[i, j, :],
                    Array2[i, j, :])
                R2 = r_value**2
                resultArray[ i , j , k] = R2

np.mean(resultArray)

0.6306478838700048

In [10]:
resultArray.shape

(14, 101, 17)